In [36]:
#!pip install pyldavis

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [37]:
import pandas as pd
import numpy as np

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import TfidfVectorizer

import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
from gensim.utils import simple_preprocess

import pyLDAvis
import pyLDAvis.gensim

import spacy

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## DATA LOADING

In [38]:
df = pd.read_csv("choiceboard_data.csv")
df.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,Subj #,Unnamed: 1,Win 1_2,Win 3_4,Win 5_6,Win 7_8,Win 9_10,Unnamed: 7,Spr 1_2,Spr 3_4,Spr 5_6,Spr 7-8,Spr 9-10,Unnamed: 13,CPD_Q1,CPD_Q2,CPD_Q3
0,1,NaN,Took the time to do some meal prep last night ...,I spent time before work to do an easy 20-minu...,Took a nap before work since my team this week...,Called a friend who happened to be on her lunc...,Took a walk outside to take in some of the rar...,NaN,Had a very busy weekend with my sister visitin...,Spent some time on the phone catching up with ...,Gave myself time to lie down on the couch and ...,Took a nap after class and ended up sleeping t...,Set aside and spent some time at night after s...,NaN,The class time devoted to health and wellness ...,During my gap years I gained a lot of insight ...,I believe self-care to be really important in ...
1,2,NaN,1. Select one of the activities that resonates...,One of the activities that I chose from the Ch...,One of the activities that I chose for self-ca...,"For my self care, I went on a walk and listene...",One of the activities that I chose for health/...,NaN,I decided to spend my self care time by going ...,"For my self care, I decided to find new music ...",I went on a run!,I watched TV with my dad!,I painted my nails! This is something I really...,NaN,The class time devoted to health and wellness ...,The way I decided which self-care to choose wa...,The time spent on my choice of self-care allow...
2,3,NaN,"Today, I cooked myself lunch and it was delici...",I took a walk around campus with my roommate t...,I listened to some of my favorite artists whil...,"Today, I spent my break eating good food and w...","Today, I spent some of my time watching a show...",NaN,I took the time to finish building my baby Yod...,I took the time to reach out and talk to one o...,This week I spent at least 20 minutes reading ...,This week I spent about an hour getting dinner...,"This weekend, I went shopping with my friend a...",NaN,The class time has reminded me on the importan...,To decide on what to do it would mostly depend...,The time spent on my choice of self care affec...
3,4,NaN,I worked on my Paint by Diamond piece.,I worked on more of the Paint by Diamonds.,I worked on my Paint by Diamond while listenin...,I worked on my Paint by Diamond while listenin...,I grabbed lunch with a friend.,NaN,I started a new Asian drama (tv show) called T...,I took a walk around my neighborhood while lis...,"I took a walk to my neighborhood park , sat on...",I went to go visit my high school teachers.,I went out and had lunch with my brother.,NaN,The class time that was devoted to health and ...,"For me, it depends on the weather à if it was ...",It made me more relaxed and less stressed abou...
4,5,NaN,I took a 20 minutes walk from the train statio...,I met up with my friends for coffee and it was...,I soaked in the bathtub for an hour with relax...,I walked around the neighborhood for thirty mi...,I tried to using peloton in our apartment bui...,NaN,Yesterday was my birthday and my boyfriend pla...,I am increasing my cardio work out in the apar...,I went to watch movie at ipic theatre in redmo...,I took a 30 minutes walk in my new neighborhoo...,It was a nice day today and my friend and I to...,NaN,I have learnt to listen to people without inte...,This is because I wanted to connect with mysel...,This has been a lifeline because I always felt...


In [39]:
df_cpd = df[["CPD_Q1", "CPD_Q3"]]
df_cpd.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CPD_Q1,CPD_Q3
0,The class time devoted to health and wellness ...,I believe self-care to be really important in ...
1,The class time devoted to health and wellness ...,The time spent on my choice of self-care allow...
2,The class time has reminded me on the importan...,The time spent on my choice of self care affec...
3,The class time that was devoted to health and ...,It made me more relaxed and less stressed abou...
4,I have learnt to listen to people without inte...,This has been a lifeline because I always felt...


In [40]:
corpus_1 = df_cpd["CPD_Q1"].dropna()
corpus_1 = corpus_1.astype(str)
corpus_1

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


0      The class time devoted to health and wellness ...
1      The class time devoted to health and wellness ...
2      The class time has reminded me on the importan...
3      The class time that was devoted to health and ...
4      I have learnt to listen to people without inte...
                             ...                        
103    It’s been a wonderful change of pace to have a...
104    The mentor team discussions with my team and p...
105    The class time devoted to health, wellness, an...
106    I personally think it was a nice break, but I ...
107    I actively disliked most of the content relate...
Name: CPD_Q1, Length: 105, dtype: object

In [41]:
#adding clearly neutral stopwords that are specific to this program or activity
stopwords = nltk.corpus.stopwords.words('english')

stopwords.append('pharmacy')
stopwords.append('pharmacist')
stopwords.append('pharmacists')
stopwords.append('health')
stopwords.append('healthcare')
stopwords.append('practitioner')
stopwords.append('mental')
stopwords.append('wellness')
stopwords.append('self-care')
stopwords.append('mentor')
stopwords.append('WIP')
stopwords.append('class')
stopwords.append('school')
stopwords.append('peers')
stopwords.append('classmates')
stopwords.append('quarter')
stopwords.append('spring')
stopwords.append('winter')
stopwords.append('fall')

stopwords = list(stopwords)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## SAMPLING

In [42]:
#using sample of words from full corpus
sample = pd.DataFrame(corpus_1).reset_index()[['CPD_Q1']]
sample['CPD_Q1'] = sample['CPD_Q1'].map(lambda x: x.lower())
sample.head()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,CPD_Q1
0,the class time devoted to health and wellness ...
1,the class time devoted to health and wellness ...
2,the class time has reminded me on the importan...
3,the class time that was devoted to health and ...
4,i have learnt to listen to people without inte...


## DATA PREPROCESSING

In [43]:
def sent_to_words(sentences):
  "gensim simple preprocess removes punctuation, lowercases, and tokenizes"
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data = sample.CPD_Q1.values.tolist()
data_words = list(sent_to_words(data))

print(data_words[:1][0][:30]) #first 30 words of the first comment

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['the', 'class', 'time', 'devoted', 'to', 'health', 'and', 'wellness', 'and', 'self', 'care', 'has', 'had', 'positive', 'effect', 'on', 'me', 'while', 'am', 'fairly', 'good', 'at', 'finding', 'pockets', 'of', 'time', 'for', 'self', 'care', 'myself']


In [44]:
#bigrams
#phrases that go together, like "new york" (bigram)

#build the models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases

#fast way to get bigrams and trigrams
bigram_mod = gensim.models.phrases.Phraser(bigram)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [45]:
nlp = spacy.load("en_core_web_sm")

# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in doc if word not in stopwords] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']): #only looking at these POS!
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [60]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words) #data_words_nostops

# Do lemmatization keeping only noun, adj, vb, adv
# data_words_bigrams == data_words => true
data_lemmatized = lemmatization(data_words, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1][0][:30]) #1st 30 words of 1st comment

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['class', 'time', 'devote', 'health', 'wellness', 'self', 'care', 'have', 'positive', 'effect', 'fairly', 'good', 'find', 'pocket', 'time', 'self', 'care', 'class', 'session', 'such', 'mentor', 'team', 'discussion', 'provide', 'space', 'engage', 'normally', 'think', 'do', 'such']


In [54]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1][0][:30])

[(0, 2), (1, 2), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 2)]


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


## BASELINE LDA MODEL

In [55]:
# Build baseline LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=2, #starting number
                                       random_state=42,
                                       chunksize=8, #of docs processed
                                       passes=100, #epochs
                                       per_word_topics=True)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [49]:
lda_model.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.061*"time" + 0.029*"care" + 0.022*"self" + 0.019*"feel" + 0.018*"take" + 0.018*"find" + 0.017*"really" + 0.017*"think" + 0.015*"well" + 0.015*"team"'),
 (1,
  '0.012*"learn" + 0.011*"team" + 0.008*"patient" + 0.008*"communication" + 0.007*"compassionate" + 0.007*"session" + 0.007*"habit" + 0.007*"skill" + 0.007*"stress" + 0.007*"other"')]

## COHERENCE MODELING

In [50]:
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('Coherence Score: ', coherence_lda)

Coherence Score:  0.3507936785919131


/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [65]:
tracker = pd.DataFrame(columns=['Topics', 'Coherence Score'])

for i in np.arange(2, 11):
  test_lda = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=i,
                                       random_state=42,
                                       passes=10)
  test_coher = CoherenceModel(model=test_lda, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  test_coher_lda = test_coher.get_coherence()
  new_row = {"Topics": i, "Coherence Score": test_coher_lda}
  tracker = tracker.append(new_row, ignore_index=True)

tracker

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<ipython-input-65-48088243941c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracker = tracker.append(new_row, ignore_index=True)
<ipython-input-65-48088243941c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracker = tracker.append(new_row, ignore_index=True)
<ipython-input-65-48088243941c>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracker = tracker.append(n

,Topics,Coherence Score
0,2.0,0.417265
1,3.0,0.385067
2,4.0,0.408965
3,5.0,0.412003
4,6.0,0.388297
5,7.0,0.371854
6,8.0,0.385236
7,9.0,0.391061
8,10.0,0.409278


In [66]:
two_topics = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=2,
                                       random_state=42,
                                       passes=10)
two_topics.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.029*"time" + 0.019*"class" + 0.016*"care" + 0.016*"health" + 0.014*"wellness" + 0.014*"self" + 0.011*"find" + 0.010*"take" + 0.010*"have" + 0.009*"quarter"'),
 (1,
  '0.039*"time" + 0.022*"class" + 0.021*"health" + 0.018*"have" + 0.017*"care" + 0.015*"wellness" + 0.015*"help" + 0.015*"think" + 0.015*"do" + 0.013*"feel"')]

These two topics have very overlapping words, so it's not ideal

In [67]:
five_topics = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=5,
                                       random_state=42,
                                       passes=10)
five_topics.print_topics()

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[(0,
  '0.020*"time" + 0.019*"self" + 0.015*"compassionate" + 0.015*"team" + 0.013*"care" + 0.013*"wellness" + 0.013*"patient" + 0.011*"class" + 0.011*"health" + 0.011*"find"'),
 (1,
  '0.031*"time" + 0.022*"health" + 0.022*"have" + 0.015*"school" + 0.015*"wellness" + 0.015*"mentor" + 0.015*"class" + 0.015*"care" + 0.014*"team" + 0.013*"other"'),
 (2,
  '0.018*"allow" + 0.016*"have" + 0.016*"time" + 0.016*"class" + 0.013*"other" + 0.011*"discussion" + 0.011*"play" + 0.011*"do" + 0.009*"team" + 0.009*"health"'),
 (3,
  '0.058*"time" + 0.033*"class" + 0.027*"care" + 0.023*"health" + 0.019*"do" + 0.019*"take" + 0.019*"have" + 0.017*"wellness" + 0.017*"self" + 0.013*"help"'),
 (4,
  '0.038*"time" + 0.024*"class" + 0.021*"help" + 0.020*"think" + 0.017*"health" + 0.016*"take" + 0.016*"care" + 0.014*"wellness" + 0.014*"break" + 0.013*"really"')]

Topic 0 - emotional: mentions compassion and patient => human-centered

Topic 1 - mentions having a mentor and a team environment

Topic 2 - allows for discussions and time to play

Topic 3 - helps a lot to have classtime for health and self-care

Topic 4 - prompts students to think more seriously about health/wellness

There are a lot of overlapping words in these topics, making these interpretations HIGHLY subjective => want to try looking at other packages that are more tuned for short text tuning

Notes (1/25):
- Try STTM? https://github.com/rwalk/gsdmm
- Visualize using PyLDADavis
- Brainstorm how I will validate